# Retrieval-Augmented Generation (RAG) Chatbot with DeepEval Evaluation – Disney Chat
	
 In this project, I created a Retrieval-Augmented Generation (RAG) chatbot to provide human responses based on real-time data from theme parks API. The RAG chatbot uses Langchain and integrates with a vector database (Qdrant) to enable retrieval and generation capabilities, uses the OpenAI GPT-3.5-turbo model and I evaluate the results using DeepEval Evaluation.

## Data Collection and Vector Database Integration

Connecting to the ThemeParks API to get data on available theme parks and their respective attractions. Each theme park has a unique ID that serves as a key for accessing multiple attraction details. 

Once the attraction data is retrieved, it is exported to an Excel file for easier visualization for this example. This step helps with data validation and allows us to get a clearer picture of the structure and contents

After exporting the data, I open the file, clear any previous data from the Qdrant vector database, and insert the new data. As this is an initial test, I’m manually deleting and uploading the new data. In a full implementation, functions would be added to detect updates automatically. Ideally, this data would be permanently stored for connections, analysis, and deeper research.


In [36]:
import requests
import pandas as pd
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from langchain.document_loaders import DataFrameLoader
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain.embeddings import OpenAIEmbeddings

# Load OpenAI Key
load_dotenv('./.env')

# get Qdrant keys form .env
qdrant_api_key = os.getenv("QDRANT_KEY")
qdrant_url_key = os.getenv("QDRANT_URL")

# Initialize the Qdrant client
qdrant_client = QdrantClient(
    url=qdrant_url_key, 
    api_key=qdrant_api_key
)

# Retrieve API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")  # Replace with your actual environment variable name
if not api_key:
    raise ValueError("The environment variable OPENAI_API_KEY is not set. Please set it in your .env file or environment.")

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# Define the base URL for the API
base_url = "https://api.themeparks.wiki/v1"

# List of entity IDs for parks
entity_ids = ['73436fe5-1f14-400f-bfbf-ab6766269e70', '9a268a42-4f13-4f7a-9c7d-d40a20d7a6eb', 'a148a943-616b-41c8-b5f8-27e67f7bdf33', '7fa8dc48-1240-4c57-b576-e4eca4bf4343', '18635b3e-fa23-4284-89dd-9fcd0aaa9c9c', 'f0ea9b9c-1ccb-4860-bfe6-b5aea7e4db2b', 'bdf9b533-144c-4b78-aa2f-5173c5ce5e85', '58392c29-d79d-49e4-9c35-0100d417d24e', '639738d3-9574-4f60-ab5b-4c392901320b', 'd21fac4f-1099-4461-849c-0f8e0d6e85a6', '28aee1df-1d05-4f53-bbf5-08f7aabff3a1', '7502308a-de08-41a3-b997-961f8275ab3c', 'abb67808-61e3-49ef-996c-1b97ed64fac6', '3ad67aac-0b97-4961-8aa8-c3b8d2350873', '18a2aeb2-7be5-4273-8cac-611ae5b519f6', '6f8764b7-172a-4fcf-8fec-10d3e44a55e4', '5f28a518-dfb4-4650-a642-2d88c8e30d8e', '99e7087b-951a-4322-84a8-572f26bc324d', '75ea578a-adc8-4116-a54d-dccb60765ef9', '47f90d2c-e191-4239-a466-5892ef59a88b', '288747d1-8b4f-4a64-867e-ea7c9b27bad8', '1c84a229-8862-4648-9c71-378ddd2c7693', 'b070cbc5-feaa-4b87-a8c1-f94cca037a18', 'ead53ea5-22e5-4095-9a83-8c29300d7c63', 'de3955e7-e3b7-4b85-884b-2d0fb2c1d71c', 'b4dd937f-a79d-4b82-922f-e8ab0fbf5b5b', 'fe78a026-b91b-470c-b906-9d2266b692da', '267615cc-8943-4c2a-ae2c-5da728ca591f', 'eb3f4560-2383-4a36-9152-6b3e5ed6bc57', 'e35cd439-ca3c-4234-ad92-d699995d0cba', '0f044655-cd94-4bb8-a8e3-c789f4eca787', 'bc4005c5-8c7e-41d7-b349-cdddf1796427', '4535960b-45fb-49fb-a38a-59cf602a0a9c', '66e12a41-3a09-40cd-8f55-8d335d9d7d93', '93142d7e-024a-4877-9c72-f8e904a37c0c', '000c724a-cd0f-41a1-b355-f764902c2b55', '67b290d5-3478-4f23-b601-2f8fb71ba803', '3cc919f1-d16d-43e0-8c3f-1dd269bd1a42', '30713cf6-69a9-47c9-a505-52bb965f01be', 'bb731eae-7bd3-4713-bd7b-89d79b031743', 'c8299e1a-0098-4677-8ead-dd0da204f8dc', 'f4bd1a23-44f0-444b-a91c-8d24f6ec5b1f', '9e938687-fd99-46f3-986a-1878210378f8', '15805a4d-4023-4702-b9f2-3d3cab2e0c1e', 'd4c88416-3361-494d-8905-23a83e9cb091', '589627eb-fe16-4373-a2db-08d73805fb1f', '0c7ab128-259a-4390-93b9-d2e0233dfc16', '95162318-b955-4b7e-b601-a99033aa0279', '75122979-ddea-414d-b633-6b09042a227c', 'dd0e159a-4e4b-48e5-8949-353794ef2ecb', '27d64dee-d85e-48dc-ad6d-8077445cd946', '9e2867f8-68eb-454f-b367-0ed0fd72d72a', '19d7f29b-e2e7-4c95-bd12-2d4e37d14ccf', '1989dca9-c8d3-43b8-b0dd-e5575f692b95', 'ca888437-ebb4-4d50-aed2-d227f7096968', 'dae968d5-630d-4719-8b06-3d107e944401', 'f9c2e042-8604-4fe8-9909-e8f95f0942f5', '32608bdc-b3fa-478e-a8c0-9dde197a4212', 'd06d91b8-7702-42c3-a8af-7d0161d471bf', '91c92c4c-e079-4488-8c99-385bc81bd5d7', '815e6367-9bbe-449e-a639-a093e216188f', 'd553882d-5316-4fca-9530-cc898258aec0', 'd67e40f9-9c02-4bfe-8ee1-b714deda9906', 'd2bef7bc-f9fc-4272-a6f1-2539d7413911', '556f0126-8082-4b66-aeee-1e3593fed188', 'c6073ab0-83aa-4e25-8d60-12c8f25684bc', '8be1e984-1e5f-40d0-a750-ce8e4dc2e87c', '3237a0c2-8e35-4a1c-9356-a319d5988e7c', 'ab49b801-9b07-4cbc-9b3e-9896e538872e', '98f634cd-c388-439c-b309-960f9475b84d', 'fc40c99a-be0a-42f4-a483-1e939db275c2', 'e9805d65-edad-4700-8942-946e6a2b4784', '164f3ee7-5fd7-47ac-addc-40b5e3e2b144', '722116aa-56be-4466-8c6f-a5acbac05da2', '66f5d97a-a530-40bf-a712-a6317c96b06d', 'a0df8d87-7f72-4545-a58d-eb8aa76f914b', '0a6123bb-1e8c-4b18-a2d3-2696cf2451f5', '24cdcaa8-0500-4340-9725-992865eb18d6', 'ddc4357c-c148-4b36-9888-07894fe75e83', '68e1d8f0-ed42-4351-af25-160421e37ce0', '7340550b-c14d-4def-80bb-acdb51d49a66', '832fcd51-ea19-4e77-85c7-75d5843b127c', 'bd0eb47b-2f02-4d4d-90fa-cb3a68988e3b', '043211c0-76f2-4456-89f8-4001be01018d', 'bb285952-7e52-4a07-a312-d0a1ed91a9ac', 'b08d9272-d070-4580-9fcd-375270b191a7', 'a4f71074-e616-4de4-9278-72fdecbdc995', '0d8ea921-37b1-4a9a-b8ef-5b45afea847b', 'ae959d1f-9fcc-4aab-8063-71e641fa57f4', '6535c36f-ea51-4156-824e-f304b27fb1f6']


# Function to fetch live data for a specific entity
def get_live_data(entity_id):
    # Construct the URL for the live data endpoint
    url = f"{base_url}/entity/{entity_id}/live"
    attractions_with_wait_time = []  # List to hold attractions data

    try:
        # Send a GET request to the API
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            # Extract live data
            live_data = data.get('liveData', [])
            # Filter attractions with available wait times
            for attraction in live_data:
                if 'queue' in attraction and 'STANDBY' in attraction['queue'] and 'waitTime' in attraction['queue']['STANDBY']:
                    # Collect relevant data
                    attraction_info = {
                        'id': attraction.get('id'),
                        'name': attraction.get('name'),
                        'wait_time': attraction['queue']['STANDBY']['waitTime'],
                        'entity_type': attraction.get('entityType'),
                        'status': attraction.get('status'),
                        'last_updated': attraction.get('lastUpdated')
                    }
                    attractions_with_wait_time.append(attraction_info)
        else:
            print(f"Error fetching data for ID {entity_id}: {response.status_code}")
    except Exception as e:
        print(f"An error occurred for ID {entity_id}: {e}")

    return attractions_with_wait_time

# Function to export data to Excel
def export_to_excel(data, filename="attractions_with_wait_times.xlsx"):
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    # Export DataFrame to an Excel file
    df.to_excel(filename, index=False, engine="openpyxl")
    print(f"Data exported to {filename}")

def fetching_live_data(all_attractions):
    #counter = 0  # Initialize a counter
    for entity_id in entity_ids:
        print(f"\nFetching live data for entity ID: {entity_id}")
        attractions = get_live_data(entity_id)
        all_attractions.extend(attractions)
    return(all_attractions)
    
def sent_to_vector():
    # Define the path to the Excel file
    excel_path = "attractions_with_wait_times.xlsx"
    # Load the Excel data into a DataFrame
    df = pd.read_excel(excel_path, engine='openpyxl')
    # Display the DataFrame to see the metadata
    docs = df[['name', 'wait_time', 'entity_type', 'status', 'last_updated']]
    
    loader = DataFrameLoader(docs, page_content_column="name")
    documents = loader.load()
    
    # In this example I'll drop all the conections to have it clear data
    #Ideally the best flow is to find and update
    #have the log of all the data for furder analysis
    
    # Get a list of all collections
    collections = qdrant_client.get_collections()
    for collection in collections.collections:
        collection_name = collection.name
        print(f"Dropping collection: {collection_name}")
        qdrant_client.delete_collection(collection_name)
    # Send to the vector db
    qdrant = Qdrant.from_documents(
        documents=documents,
        embedding=embeddings,
        url=qdrant_url_key,
        collection_name="chatbot",
        api_key=qdrant_api_key
    )
    print(f"Sucess sent to vector")
    
# Main function to fetch data and export it
def main():
    all_attractions = []  # List to hold all attractions from all parks
    fetching_live_data(all_attractions)
    # Export collected data to Excel
    if all_attractions:
        export_to_excel(all_attractions)
        sent_to_vector()
    else:
        print("No attractions with wait times found.")
        
# Execute the main function
if __name__ == "__main__":
    main()


Fetching live data for entity ID: 73436fe5-1f14-400f-bfbf-ab6766269e70

Fetching live data for entity ID: 9a268a42-4f13-4f7a-9c7d-d40a20d7a6eb

Fetching live data for entity ID: a148a943-616b-41c8-b5f8-27e67f7bdf33

Fetching live data for entity ID: 7fa8dc48-1240-4c57-b576-e4eca4bf4343

Fetching live data for entity ID: 18635b3e-fa23-4284-89dd-9fcd0aaa9c9c

Fetching live data for entity ID: f0ea9b9c-1ccb-4860-bfe6-b5aea7e4db2b

Fetching live data for entity ID: bdf9b533-144c-4b78-aa2f-5173c5ce5e85

Fetching live data for entity ID: 58392c29-d79d-49e4-9c35-0100d417d24e

Fetching live data for entity ID: 639738d3-9574-4f60-ab5b-4c392901320b

Fetching live data for entity ID: d21fac4f-1099-4461-849c-0f8e0d6e85a6

Fetching live data for entity ID: 28aee1df-1d05-4f53-bbf5-08f7aabff3a1

Fetching live data for entity ID: 7502308a-de08-41a3-b997-961f8275ab3c

Fetching live data for entity ID: abb67808-61e3-49ef-996c-1b97ed64fac6

Fetching live data for entity ID: 3ad67aac-0b97-4961-8aa8-c3b8d

### Integrating Langchain and Querying with OpenAI

In this part, I integrate OpenAI using Langchain to help create an interface that can interact with the Qdrant vector database. Langchain make the connection between OpenAI’s API and the vector database easier, allowing more classes and functions. 


In [37]:
import openapi_client
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
import os
from dotenv import load_dotenv

# Load OpenAI Key
load_dotenv('./.env')

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

#choosing the model
chat = ChatOpenAI(
    model='gpt-3.5-turbo'
)

# Function to send a prompt and receive a response
def ask_openai(prompt):
    response = chat(prompt)
    return response

# Test the function with a sample question
question = "What is the waiting time for Safari World and Jurassic World Adventure? each one is more fun?"
#question = "What is the waiting time for T Express? is it fun?"
response = ask_openai(question)
print("Response:", response)

Response: content="The waiting time for Safari World and Jurassic World Adventure may vary depending on the day and time you visit. It is recommended to check the theme park's website or ask a staff member for the most up-to-date information on waiting times.\n\nBoth Safari World and Jurassic World Adventure are popular attractions and offer different experiences. Safari World allows you to see various animals up close in a safari-like setting, while Jurassic World Adventure takes you on a thrilling journey through a prehistoric world with animatronic dinosaurs.\n\nUltimately, the level of fun you have at each attraction will depend on your personal preferences. Some may enjoy the excitement of Jurassic World Adventure, while others may prefer the more leisurely experience of Safari World. It is recommended to try both and see which one you enjoy more!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 26, 'total_tokens': 

### Fine-tune Qdrant responses
In this phase, we fine-tune Qdrant responses by adding context around user questions, enabling richer, more informative answers while minimizing hallucinations. This approach structures responses to feel more conversational and aligned with user expectations.

In [38]:

import openapi_client
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
import os

# Load OpenAI Key
load_dotenv('./.env')

# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
qdrant_api_key = os.getenv("QDRANT_KEY")
qdrant_url_key = os.getenv("QDRANT_URL")

#choosing the model
chat = ChatOpenAI(
    model='gpt-3.5-turbo'
)

# Initialize the Qdrant client
qdrant_client = QdrantClient(
    url=qdrant_url_key, 
    api_key=qdrant_api_key
)

# Initialize the Qdrant vector store
qdrant = Qdrant(
    client=qdrant_client,
    collection_name="chatbot",  
    embeddings=embeddings
)


# Custom prompt function to include wait times in the response
def custom_prompt(query: str):
    # Perform similarity search and retrieve relevant documents
    results = qdrant.similarity_search(query, k=5)
    
    # Helper function to format wait times
    def format_wait_time(wait_time):
        try:
            # Convert wait time to an integer and handle hours and minutes
            wait_time = int(wait_time)
            hours, minutes = divmod(wait_time, 60)
            if hours > 0:
                return f"{hours} hours and {minutes} minutes" if minutes else f"{hours} hours"
            else:
                return f"{minutes} minutes"
        except (ValueError, TypeError):
            return "Not available"

    # Format the source knowledge to include attraction names and wait times
    source_knowledge = "\n".join([
        f"Attraction: {x.page_content}, Wait Time: {format_wait_time(x.metadata.get('wait_time'))}"
        for x in results
    ])
    
    # Construct the augmented prompt using the extracted knowledge
    augment_prompt = f"""Using the contexts below, answer the query in two parts: 

    First, answer the question about wait times, Then answer the question about which attraction might be more fun. White a cool sentence

    Contexts:
    {source_knowledge}

    Query: {query}"""
    
    return augment_prompt

# Example query to test the function
query = "What is the waiting time for Safari World and Jurassic World Adventure? each one is more fun?"
#question = "What is the waiting time for T Express? is it fun?"
print(custom_prompt(query))

Using the contexts below, answer the query in two parts: 

    First, answer the question about wait times, Then answer the question about which attraction might be more fun. White a cool sentence

    Contexts:
    Attraction: Jurassic World Adventure, Wait Time: 15 minutes
Attraction: Safari World, Wait Time: 1 hours and 30 minutes
Attraction: Disney's Animal Kingdom Theme Park, Wait Time: Not available
Attraction: Kilimanjaro Safaris, Wait Time: Not available
Attraction: Splash Safari, Wait Time: Not available

    Query: What is the waiting time for Safari World and Jurassic World Adventure? each one is more fun?


### Human-like Response with the LLM + Vector db
At this stage, I export the responses to evaluate their quality, refining them to sound more natural and human-like. I focus on clarity and conversational flow, with iterative testing to ensure that responses are accurate and consistently meet our quality standards

In [39]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
messages = [
    SystemMessage(content="You are a knowledgeable assistant that provides real-time information about attractions, wait times, and special events at various theme parks."),
    HumanMessage(content="Hi AI, can you give me live updates on attractions?"),
    AIMessage(content="Certainly! I can provide you with current wait times and updates on any attraction. Just let me know which one you're interested in.")
]

prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

The current wait time for Safari World is 1 hour and 30 minutes. For Jurassic World Adventure, the wait time is 15 minutes. If you're looking for an exhilarating adventure with lifelike dinosaurs, Jurassic World Adventure might be more fun for you.


### Quality Assurance with DeepEval
Through DeepEval, I can measure the accuracy of responses and detect any deviations from factual information. This feedback is invaluable in fine-tuning the model to maintain high standards of reliability, ensuring that users receive correct and relevant information!

In [41]:
# Import necessary modules from the DeepEval library
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric

# Initialize the metrics for evaluating answer relevancy and faithfulness
answer_relevancy = AnswerRelevancyMetric()
faithfulness = FaithfulnessMetric()

# Create a test case with the input question, actual output, and expected output for comparison
test_case = LLMTestCase(
    input="What is the waiting time for Safari World and Jurassic World Adventure? each one is more fun?",
    actual_output="The current wait time for Safari World is 1 hour and 30 minutes. For Jurassic World Adventure, the wait time is 15 minutes. If you're looking for an exhilarating adventure with lifelike dinosaurs, Jurassic World Adventure might be more fun for you.",
    expected_output=" Safari World is X hours and X minutes, and the wait time for Jurassic World Adventure is X hour and X minutes.",
    retrieval_context=[
        """Jurassic World Adventure has an immersive dinosaur experience, Safari World is a thrill"""
    ]
)

# Measure answer relevancy
answer_relevancy.measure(test_case)
print("Answer Relevancy Score: ", answer_relevancy.score)
print("Answer Relevancy Reason: ", answer_relevancy.reason)

# Measure faithfulness
faithfulness.measure(test_case)
print("Faithfulness Score: ", faithfulness.score)
print("Faithfulness Reason: ", faithfulness.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

None

Score:  1.0
Reason:  The score is 1.00 because the answer is perfectly relevant with no irrelevant statements. Great job!


Score:  1.0
Reason:  The score is 1.00 because there are no contradictions, indicating a perfect alignment between the actual output and the retrieval context. Great job!
